In [1]:
import numpy as np
from scipy import stats
import os
import pandas as pd
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import KFold
import math
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import sqlite3
%matplotlib inline

# read in data and labels
data = pd.read_csv("updated_data.csv", low_memory=False)
# labels = pd.read_cvs('lablels.csv') #### generate the labels and save..., isolate column from data and delete rows accordingly

# filter out the bad schools (rows)
data_filtered = []
for i in range(0, len(data)):
    if (data.ix[i, 'DISTANCEONLY'] == "Not distance-education only") and \
        ((data.ix[i, 'HIGHDEG'] == "Graduate degree") or (data.ix[i, 5] == "Bachelor's degree")) and \
        (data.ix[i, 'CURROPER'] == "Currently certified as operating"):
        data_filtered.append(data.ix[i])
data = pd.DataFrame(data_filtered, columns=data.columns.values)
data_final = data.drop(['Unnamed: 0', 'DISTANCEONLY', 'HIGHDEG', 'CURROPER'], axis=1)
data_final = data_final.reset_index()
data_final = data_final.drop('index',axis=1)

In [9]:
# drop rows with NaNs and privacy suppressed data
to_drop = []
for i in range(100): # EDITTEDDDDD
    print(i)
    for j in range(0, len(data_final.columns)):
        if data_final.ix[i, j] == -1 or data_final.ix[i, j] == "PrivacySuppressed":
            to_drop.append(i)
            break
data_processed = data_final.drop(data_final.index[to_drop])
# data_processed = data_processed.reset_index()
# data_processed = data_processed.drop('index',axis=1)
data_processed

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


,Year,UNITID,INSTNM,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,...,md_earn_wne_p6,pct10_earn_wne_p6,pct25_earn_wne_p6,pct75_earn_wne_p6,pct90_earn_wne_p6,sd_earn_wne_p6,gt_25k_p6,mn_earn_wne_inc1_p6,mn_earn_wne_inc2_p6,mn_earn_wne_inc3_p6
96,2013,110307,Phillips Graduate Institute,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,2013,110398,University of California-Hastings College of Law,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,104600,33700,75200,159000,207300,64800,0.930000007153,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed
101,2013,110404,California Institute of Technology,977,0.3040,0.0174,0.1003,0.4248,0.0031,0.0000,...,62100,8500,23500,86100,99800,46600,0.742857158184,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed
102,2013,110413,California Lutheran University,2882,0.5232,0.0427,0.2523,0.0642,0.0135,0.0087,...,45100,14200,29800,60900,81400,30900,0.798977851868,45000.0,50600.0,47000.0
103,2013,110422,California Polytechnic State University-San Lu...,18664,0.5956,0.0076,0.1489,0.1134,0.0024,0.0027,...,49800,16000,32400,68100,86700,36800,0.825837314129,50800.0,52100.0,53200.0
104,2013,110468,Alliant International University,254,0.3150,0.1142,0.2638,0.0236,0.0000,0.0000,...,46800,15600,31400,62900,82800,26800,0.817351579666,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed
105,2013,110486,California State University-Bakersfield,7226,0.1960,0.0656,0.4979,0.0650,0.0073,0.0017,...,40600,9400,23400,55500,72200,24800,0.729636073112,40200.0,41800.0,45900.0
106,2013,110495,California State University-Stanislaus,7751,0.2820,0.0255,0.4496,0.1133,0.0035,0.0072,...,39100,9500,23800,50700,64000,21900,0.731944441795,37600.0,39400.0,40300.0
107,2013,110510,California State University-San Bernardino,15887,0.1743,0.0729,0.5549,0.0632,0.0025,0.0023,...,41100,9700,22500,56600,71700,26200,0.714063704014,38700.0,43900.0,46100.0
108,2013,110529,California State Polytechnic University-Pomona,20857,0.2159,0.0323,0.3722,0.2501,0.0020,0.0028,...,44100,11900,25900,61600,79200,27700,0.762355744839,44500.0,47000.0,48600.0


In [8]:
data_final

,Year,UNITID,INSTNM,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,...,md_earn_wne_p6,pct10_earn_wne_p6,pct25_earn_wne_p6,pct75_earn_wne_p6,pct90_earn_wne_p6,sd_earn_wne_p6,gt_25k_p6,mn_earn_wne_inc1_p6,mn_earn_wne_inc2_p6,mn_earn_wne_inc3_p6
0,2013,100654,Alabama A & M University,4051,0.0279,0.9501,0.0089,0.0022,0.0012,0.0010,...,25700,6400,14600,38900,53000,34500,0.513227522373,28200.0,30300.0,30900.0
1,2013,100663,University of Alabama at Birmingham,11200,0.5987,0.2590,0.0258,0.0518,0.0026,0.0007,...,35100,10600,23200,49800,69100,27900,0.724107146263,38800.0,39700.0,40600.0
2,2013,100690,Amridge University,322,0.2919,0.4224,0.0093,0.0031,0.0031,0.0031,...,40600,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,33900,0.677777767181,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed
3,2013,100706,University of Alabama in Huntsville,5525,0.7012,0.1310,0.0338,0.0364,0.0145,0.0002,...,37900,10800,23000,56600,70700,26700,0.720472455025,37900.0,41900.0,46100.0
4,2013,100724,Alabama State University,5354,0.0161,0.9285,0.0114,0.0015,0.0009,0.0007,...,22600,4300,13200,33800,42500,15600,0.44180226326,22700.0,26300.0,27800.0
5,2013,100751,The University of Alabama,28692,0.7865,0.1140,0.0313,0.0112,0.0044,0.0011,...,35900,12300,23800,49300,65200,26100,0.729986429214,35900.0,40200.0,39700.0
6,2013,100812,Athens State University,2999,0.7513,0.1064,0.0213,0.0047,0.0190,0.0007,...,34900,10600,23300,42300,58500,20700,0.717981874943,31700.0,37900.0,36800.0
7,2013,100830,Auburn University at Montgomery,4322,0.5532,0.3031,0.0079,0.0245,0.0037,0.0000,...,31400,9700,19700,41700,55700,20100,0.646017670631,30400.0,34300.0,37300.0
8,2013,100858,Auburn University,19761,0.8543,0.0714,0.0253,0.0213,0.0077,0.0000,...,39500,14700,27400,53400,70100,44700,0.789690136909,44900.0,44600.0,42900.0
9,2013,100937,Birmingham Southern College,1181,0.8103,0.0940,0.0305,0.0390,0.0102,0.0000,...,34200,9700,22900,47600,61100,26800,0.704347848892,35700.0,37000.0,38100.0


In [ ]:
data_processed.to_csv('data_processed.csv', encoding='utf-8')




In [ ]:
# normalize the data column(feature)-wise by using z-scores
stats.zscore(data, axis=1)

def plotPred(y_predict, y_test, name):
    plt.figure()
    plt.scatter(y_test, y_predict)
    t = np.arange(0, 100000, 1)
    plt.plot(t,t)
    plt.xlabel('Actual incomes')
    plt.ylabel('Predicted incomes')
    plt.title(name)
    plt.show()
    print("RMSE:", rmse(y_test, y_predict))    
    print("r2:", r2_score(y_test, y_predict))

def plotResiduals(y_predict, y_test, name):
    plt.figure()
    plt.hist(y_predict-y_test)
    plt.xlabel('Residual')
    plt.ylabel('Frequency')
    plt.title(name + ' Residuals')
    plt.show()

# http://stackoverflow.com/questions/17197492/root-mean-square-error-in-python
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def listTenLargestResiduals(y_predict, y_test) 
	residuals = list(y_predict-y_test).sort()
	for x in range(10):
		print(residuals[i])

#### gridsearch hyperparameter optimization
# evaluate with 3-fold finetuning 

# fit and test the regressor, output graph and evaluation statistics
def regress(model, name):
    reg = model
    reg.fit(x_train, y_train)
    y_predict = reg.predict(x_test)
    plotPred(y_predict, y_test, name)
    plotResiduals(y_predict, y_test, name)
    listTenLargestResiduals(y_predict, y_test)

# test different regression models - still need to finetune
regress(linear_model.LinearRegression(), 'Ordinary Least Squares Regression')
# regress(linear_model.Ridge(), 'Ridge Regression')
# regress(linear_model.Lasso(alpha = 0.0001), 'Lasso Regression')
# regress(linear_model.ElasticNet(alpha=.0006), 'Elastic Net Regression')
# regress(DecisionTreeRegressor(max_depth=10), 'Decision Tree Regression')
# regress(RandomForestRegressor(n_estimators = 100, max_depth = 5, warm_start = False), 'Random Forest Regression')
# regress(KNeighborsRegressor(), "KNN Regressor")
# regress(SVR(C=10, kernel='linear'), 'Linear Support Vector Regression')... takes a long time

# feature importance using random forest

# output covariance matrix heat map

# PCA plots

